# Scroller

Scrollers只是随着时间的推移，追踪滚动的偏移量，但它们不会自动地把这些位置应用到view上。**我们应该按一定频率，获取并应用这些新的坐标值，来让滚动动画更加顺滑。**

> 参考 https://juejin.im/post/6844903791066628110

# Scroller的典型用法

通过 invalidate() 驱动

```java
Scroller mScroller = new Scroller(mContext);

private void smoothScroll(int destX, int destY) {
        int scrollX = getScrollX();
        int deltaX = destX - scrollX;
        mScroller.startScroll(scrollX, 0, deltaX, 0, 500);
        invalidate();
    }

@Override
    public void computeScroll() {
        super.computeScroll();
        if (mScroller.computeScrollOffset()) {
            scrollTo(mScroller.getCurrX(), mScroller.getCurrY());
            invalidate();
        }
    }
```

// 参考 : https://www.jianshu.com/p/57ce979b23e8

# 获取配置参数
```java
final ViewConfiguration vc = ViewConfiguration.get(context);
mTouchSlop = vc.getScaledTouchSlop();
mMinFlingVelocity = vc.getScaledMinimumFlingVelocity();
mMaxFlingVelocity = vc.getScaledMaximumFlingVelocity();
DisplayMetrics metric = context.getResources().getDisplayMetrics();
SCREEN_WIDTH = metric.widthPixels;
SCREEN_HEIGHT = metric.heightPixels;
```

> https://github.com/zincPower/UI2018

# VelocityTracker

所以我们需要先借住 VelocityTracker 进行获取我们当前手指的滑动速度，但这里需要注意的是，要限制其最大和最小速度。因为速度过快和过慢，都会导致交互效果不佳。

```java
mMaximumVelocity = ViewConfiguration.get(context).getScaledMaximumFlingVelocity();
mMinimumVelocity = ViewConfiguration.get(context).getScaledMinimumFlingVelocity();
```

# 惯性滑动效果

## 获取抬起时的速度

```kotlin
 /**
* 速度追踪
*/
private var velocityTracker: VelocityTracker? = null
private val maxVelocity = ViewConfiguration.get(context).scaledMaximumFlingVelocity
private val minVelocity = ViewConfiguration.get(context).scaledMinimumFlingVelocity

     fun handleEvent(event: MotionEvent): Boolean {

            if (velocityTracker == null) {
                velocityTracker = VelocityTracker.obtain()
            }
            velocityTracker!!.addMovement(event)

            when (event.action) {
                MotionEvent.ACTION_DOWN -> {
                    // ...
                    return true
                }
                MotionEvent.ACTION_MOVE -> {
                    // ...
                    return true
                }
                MotionEvent.ACTION_UP -> {
                    // ...
                    // 计算手抬起时的速度
                    velocityTracker!!.computeCurrentVelocity(1000, maxVelocity.toFloat())
                    val velocityY = velocityTracker!!.yVelocity
                    if (abs(velocityY) > minVelocity) {
                        // 如果满足惯性滑动的最小速度
                        start(velocityY)
                    } else {
                        // ...
                    }
                    velocityTracker?.recycle()
                    velocityTracker = null
                    return true
                }
                else -> {
                }
            }
            return false
        }
```

## 使用Scroller#Fling滑动

```kotlin
       /**
         * 开始滚动
         */
        private fun start(velocityY: Float) {
            preMoveY = moveY
            // 先停止上一次的滚动
            if (!scroller.isFinished) {
                scroller.abortAnimation()
            }

            // 触发 fling
            scroller.fling(
                0, 0,
                0, velocityY.toInt(),
                0, 0,
                -maxScrollY, maxScrollY
            )

            // 开启滚动循环, 根据scroller的偏移量刷新界面
            post(this)
        }

        override fun run() {
            moveY = preMoveY + scroller.currY.toFloat()
            invalidate()

            if (scroller.computeScrollOffset()) {
                post(this)
                return
            } else {
                // 滚动已经结束
                handleValueSelect()
            }
        }

        private fun stop() {
            if (!scroller.isFinished)
                scroller.abortAnimation()
        }
```

# demo

ScrollSelectView.kt

```kotlin
import android.annotation.SuppressLint
import android.content.Context
import android.graphics.Canvas
import android.graphics.Color
import android.graphics.Paint
import android.util.AttributeSet
import android.util.Log
import android.view.MotionEvent
import android.view.VelocityTracker
import android.view.View
import android.view.ViewConfiguration
import android.view.animation.DecelerateInterpolator
import android.widget.Scroller
import java.util.*
import kotlin.math.abs
import kotlin.math.min
import kotlin.math.pow


/**
 * by ganxiao
 */
class ScrollSelectView @JvmOverloads constructor(
    context: Context?,
    attrs: AttributeSet? = null,
    defStyleAttr: Int = 0
) : View(context, attrs, defStyleAttr) {
    /**
     * 默认宽度
     */
    private val defWidth = 80.dpToPx()

    /**
     * 默认高度
     */
    private val defHeight = 300.dpToPx()

    /**
     * 选择回调
     */
    var listener: OnValueChangeListener? = null

    /**
     * 布长
     */
    private var step = 1

    /**
     * 滑动的下标，真实值为该值对取值范围取余
     */
    private var index = 0

    /**
     * 选中行文字大小
     */
    private var textSize = 45.dpToPx()

    /**
     * 未选中行文字颜色
     */
    private val textNormalColor = Color.WHITE

    /**
     * 选中行文字颜色
     */
    private val textActiveColor = 0xffFFB27D.toInt()

    private val mPaint = Paint(Paint.ANTI_ALIAS_FLAG)

    /**
     * 最小值
     */
    private var minValue = 0

    /**
     * 最大值
     */
    private var maxValue = 99

    /**
     * 滑动时手指的Y轴移动距离
     */
    private var moveY = 0f

    /**
     * 每一行文字内容占据的高度
     */
    private var lineHeight = 0f

    /**
     * 高亮的数字上还要绘制的个数
     */
    private val topN = 800

    /**
     * 高亮的数字上显示的个数
     */
    private val topNShow = 2

    /**
     * 缩放系数
     */
    private val zoomFactor = 2f

    /**
     * 放置选中 value 的数组
     */
    private val showValueList: MutableList<Int> = ArrayList()

    /**
     * 选择部份是否包含最大值
     */
    private var includeMax = false

    /**
     * 滑动帮助类
     */
    private val scrollHelper = ScrollHelper()

    init {
        refreshShowValueList()
    }

    fun debug(msg: String) {
        Log.d("ScrollSelectView", msg)
    }

    override fun onMeasure(widthMeasureSpec: Int, heightMeasureSpec: Int) {
        setMeasuredDimension(
            resolveSize(defWidth, widthMeasureSpec),
            resolveSize(defHeight, heightMeasureSpec)
        )
    }

    @SuppressLint("ClickableViewAccessibility")
    override fun onTouchEvent(event: MotionEvent): Boolean {
        val result = scrollHelper.handleEvent(event)
        return if (result)
            result
        else
            super.onTouchEvent(event)
    }

    override fun onDraw(canvas: Canvas) {
        lineHeight = height / (1f + 2f * topNShow)
        mPaint.textSize = textSize.toFloat()
        var startY = moveY + (0 - lineHeight * (topN - topNShow))
        debug("moveY: " + moveY)
        for (i in 0 until 1 + topN * 2) {
            val centerY = startY + lineHeight / 2f
            val drawIndex = index + i - topN
            val drawValue = getValueFromIndex(drawIndex)
            if (centerY > lineHeight * topNShow && centerY < lineHeight * (topNShow + 1)) {
                mPaint.color = textActiveColor
            } else {
                mPaint.color = textNormalColor
            }
            canvas.save()
            // val scale = calScale(centerY)
            val scale = 1f
            canvas.scale(scale, scale, 0f, centerY)
            val text = drawValue.format02d()
            mPaint.textAlign = Paint.Align.CENTER
            val fontMetrics = mPaint.fontMetrics
            val tmp = abs(fontMetrics.top) + abs(fontMetrics.bottom) / 2f - abs(fontMetrics.bottom)
            canvas.drawText(text, width * 0.5f / scale, centerY + tmp, mPaint)
            canvas.restore()
            startY += lineHeight
        }
    }

    /**
     * 缩放公式
     */
    private fun calScale(y: Float): Float {
        var result = 0.2f
        if (y <= 0 || y > height) {
            return result
        }
        val t = y / height
        result = (1 - (zoomFactor * (t - 0.5f).toDouble()).pow(2.0)).toFloat()
        val minScale = 0.7f
        return min(minScale, result)
    }

    var value: Int
        get() = getValueFromIndex(index)
        set(value) {
            for (i in showValueList.indices) {
                if (value == showValueList[i]) {
                    index = i
                    refreshShowValueList()
                    invalidate()
                    return
                }
            }
        }

    private fun refreshShowValueList() {
        showValueList.clear()
        if (includeMax) {
            var value = minValue
            while (value <= maxValue) {
                showValueList.add(value)
                value += step
            }
        } else {
            var value = minValue
            while (value < maxValue) {
                showValueList.add(value)
                value += step
            }
        }
    }

    /**
     * 根据当前的偏移量确定用户选择的 value
     */
    private fun handleValueSelect() {
        var absY = abs(moveY)
        val direction = if (moveY > 0) -1 else 1
        if (absY > lineHeight / 2f) {
            var moveIndex = 1
            absY -= lineHeight / 2f
            moveIndex += (absY / lineHeight).toInt()
            index += moveIndex * direction
            listener?.onValueChange(value)
        }
        invalidate()
        moveY = 0f
    }

    private fun getValueFromIndex(i: Int): Int {
        var i = i
        if (maxValue <= minValue) {
            return minValue
        }
        if (showValueList.size == 0) {
            return minValue
        }
        if (i < 0) {
            i = showValueList.size + i % showValueList.size
        }
        val value = i % showValueList.size
        return showValueList[value]
    }

    inner class ScrollHelper : Runnable {

        /**
         * 速度追踪
         */
        private var velocityTracker: VelocityTracker? = null
        private val maxVelocity = ViewConfiguration.get(context).scaledMaximumFlingVelocity
        private val minVelocity = ViewConfiguration.get(context).scaledMinimumFlingVelocity
        private val scroller = Scroller(context, DecelerateInterpolator())
        private var startY = 0f
        private val maxScrollY = 405.dpToPx()
        private var preMoveY = 0f

        fun handleEvent(event: MotionEvent): Boolean {

            if (velocityTracker == null) {
                velocityTracker = VelocityTracker.obtain()
            }
            velocityTracker!!.addMovement(event)

            when (event.action) {
                MotionEvent.ACTION_DOWN -> {
                    scrollHelper.stop()
                    startY = event.y
                    return true
                }
                MotionEvent.ACTION_MOVE -> {
                    moveY = event.y - startY
                    invalidate()
                    return true
                }
                MotionEvent.ACTION_UP -> {
                    moveY = event.y - startY
                    // 计算手抬起时的速度
                    velocityTracker!!.computeCurrentVelocity(1000, maxVelocity.toFloat())
                    val velocityY = velocityTracker!!.yVelocity
                    debug("velocityY: $velocityY")
                    if (abs(velocityY) > minVelocity) {
                        start(velocityY)
                    } else {
                        handleValueSelect()
                    }
                    velocityTracker?.recycle()
                    velocityTracker = null
                    return true
                }
                else -> {
                }
            }
            return false
        }

        /**
         * 开始滚动
         */
        private fun start(velocityY: Float) {
            preMoveY = moveY
            // 先停止上一次的滚动
            if (!scroller.isFinished) {
                scroller.abortAnimation()
            }

            // 触发 fling
            scroller.fling(
                0, 0,
                0, velocityY.toInt(),
                0, 0,
                -maxScrollY, maxScrollY
            )

            // 开启滚动循环, 根据scroller的偏移量刷新界面
            post(this)
        }

        override fun run() {
            moveY = preMoveY + scroller.currY.toFloat()
            debug("run moveY: $moveY")
            invalidate()

            if (scroller.computeScrollOffset()) {
                post(this)
                return
            } else {
                // 滚动已经结束
                handleValueSelect()
            }
        }

        private fun stop() {
            if (!scroller.isFinished)
                scroller.abortAnimation()
        }

    }

    interface OnValueChangeListener {
        fun onValueChange(value: Int)
    }

}
```